In [206]:
import pandas as pd
import plotly as p
import plotly.express as px 
import plotly.io as pio
import plotly.graph_objects as go
import numpy as np
pio.templates.default = "plotly_dark"


In [ ]:
denied_boarding_overall = pd.read_csv('..\\raw_data\\DeniedBoardingOverall_1990_2021.csv')
denied_boarding_overall = denied_boarding_overall[0:5].T.reset_index()
denied_boarding_overall = denied_boarding_overall[1:].rename(columns = {'index':'year', 0:'boarded', 1:'denied_boarding', 2:'voluntary', 3:'involuntary', 4:'perc_denied_boarding'})

denied_boarding_overall['boarded']  = pd.to_numeric(denied_boarding_overall['boarded'].str.replace(",","") )*1000
denied_boarding_overall['denied_boarding']  = pd.to_numeric(denied_boarding_overall['denied_boarding'].str.replace(",","") )*1000
denied_boarding_overall['voluntary']  = pd.to_numeric(denied_boarding_overall['voluntary'].str.replace(",","") )*1000
denied_boarding_overall['involuntary']  = pd.to_numeric(denied_boarding_overall['involuntary'].str.replace(",","") )*1000
denied_boarding_overall['perc_denied_boarding']  = pd.to_numeric(denied_boarding_overall['perc_denied_boarding'] )

denied_boarding_overall.loc[denied_boarding_overall['year'] == "(R) 2020", ['year'] ]= '2020'

denied_boarding_overall['perc_denied_boarding_voluntary'] = round((denied_boarding_overall['voluntary']/denied_boarding_overall['boarded'])*100,4)
denied_boarding_overall['perc_denied_boarding_involuntary'] = round((denied_boarding_overall['involuntary']/denied_boarding_overall['boarded'])*100,4)

In [225]:
denied_boarding = pd.read_csv('..\\raw_data\\Denied-Confirmed-Space-2021-Marketing-Carrier-Q1.csv')
denied_boarding = denied_boarding.iloc[:][0:20] # remove column definitions

denied_boarding.columns = ['carrier', 
                           'num_passengers_denied_boarding_involuntarily_1a', 
                           'num_passengers_denied_boarding_involuntarily_1b', 
                           'num_passengers_denied_boarding_involuntarily_2a', 
                           'num_passengers_denied_boarding_involuntarily_2b',
                           'num_passengers_denied_boarding_involuntarily_2c',
                           'num_passengers_denied_boarding_involuntarily_total',
                           'num_passengers_denied_boarding_involuntarily_and_received_comp_total',
                           'num_passengers_voluntarily_gave_up_seat',
                           'total_boardings', 
                           'total_comp_paid_to_passengers_voluntarily_gave_up_seat']

""" 1.  Number of passengers who were denied boarding involuntarily from flights that were oversold, and:										
(a)  who qualified for denied boarding compensation within the meaning of § 250.5(a)(2) and 250.5(b)(2)										
(b) who qualified for denied boarding compensation within the meaning of § 250.5(a)(3) and 250.5(b)(3) 										
2.  Number of passengers denied boarding involuntarily from flights that were oversold, who did not qualify for denied boarding compensation due to:										
(a) The passenger does not comply fully with the carrier's contract of carriage or tariff provisions regarding ticketing, reconfirmation, check-in, and acceptability for transportation (see § 250.6(a)) 										
(b) substitution of aircraft of lesser capacity or due to weight/balance restrictions on an aircraft with a designed passenger capacity of 60 or fewer seats (see § 250.6(b))										
(c) The carrier arranges comparable air transportation or other transportation that is planned to arrive not later than 1 hour after the planned arrival time of the passenger's original flight or flights (see § 250.6(d))carriage										
3.  TOTAL NUMBER DENIED BOARDING INVOLUNTARILY										
4.  Number of passengers denied boarding involuntarily from an oversold flight who actually received compensation, regardless of the type of compensation (e.g., voucher, cash).										
5.  Number of passengers who voluntarily accepted a carrier’s offer to give up reserved space due to a potential oversale situation and did not travel on their original flight in exchange for a payment of the carrier’s choosing.										
6.  Total Boardings										
7.  Amount of compensation paid to passengers who voluntarily accepted a carrier’s offer to give up reserved space on an oversold flight that received cash or cash equivalent payment.										
"""

denied_boarding['num_passengers_denied_boarding_involuntarily_1'] = denied_boarding['num_passengers_denied_boarding_involuntarily_1a']+denied_boarding['num_passengers_denied_boarding_involuntarily_1b']
denied_boarding['num_passengers_denied_boarding_involuntarily_2'] = denied_boarding['num_passengers_denied_boarding_involuntarily_2a']+denied_boarding['num_passengers_denied_boarding_involuntarily_2b']+denied_boarding['num_passengers_denied_boarding_involuntarily_2c']
denied_boarding['year'] = 2021
denied_boarding['quarter'] = 1

denied_boarding['total_boardings']  = denied_boarding['total_boardings'].str.replace(",","")  
denied_boarding['total_boardings']  = pd.to_numeric(denied_boarding['total_boardings'])

denied_boarding['num_passengers_voluntarily_gave_up_seat']  = denied_boarding['num_passengers_voluntarily_gave_up_seat'].str.replace(",","")  
denied_boarding['num_passengers_voluntarily_gave_up_seat']  = pd.to_numeric(denied_boarding['num_passengers_voluntarily_gave_up_seat'])

denied_boarding['perc_denied_boarding'] = round(((denied_boarding['num_passengers_denied_boarding_involuntarily_total']+denied_boarding['num_passengers_voluntarily_gave_up_seat'])/denied_boarding['total_boardings'])*100, 6)
denied_boarding['perc_denied_boarding_voluntarily'] = round((denied_boarding['num_passengers_voluntarily_gave_up_seat']/denied_boarding['total_boardings'])*100, 6)
denied_boarding['perc_denied_boarding_involuntarily'] = round((denied_boarding['num_passengers_denied_boarding_involuntarily_total']/denied_boarding['total_boardings'])*100, 6)

denied_boarding = denied_boarding[denied_boarding['carrier'].isin(['Alaska Airlines Network',
                                                 "Allegiant Airlines",
                                                 "American Airlines Network",
                                                 "Delta Air Lines Network",
                                                 "Frontier Airlines",
                                                 "Hawaiian Airlines Network",
                                                 "JetBlue Airways",
                                                 "Southwest Airlines",
                                                 "Spirit Airlines",
                                                 "United Airlines Network"])]

In [199]:
fig = px.bar(denied_boarding.sort_values('perc_denied_boarding', ascending=False).rename(columns = { "perc_denied_boarding_voluntarily":"Voluntary", 'perc_denied_boarding_involuntarily':"Involuntary"}), 
             x="carrier", y=['Voluntary', "Involuntary"], 
                labels={
                    "carrier": "Airline Carrier",
                     "value":"Percent of Passengers Denied Boarding", 
                     "variable":"Denial Type"
                 },
                title="Percentage of Passengers Denied Boarding by Airline Carrier", 
                color_discrete_sequence=px.colors.qualitative.Dark24)
fig.update_xaxes(tickangle = 45)
fig.show()


In [177]:

fig = px.scatter(denied_boarding, x="total_boardings", y="num_passengers_denied_boarding_involuntarily_total",
	         size="perc_denied_boarding", color="carrier",
                 hover_name="carrier", log_x=True, size_max=60, 
                 labels={
                     "total_boardings": "Total Boardings",
                     "num_passengers_denied_boarding_involuntarily_total": "Total Passengers Denied Boarding",
                     "carrier": "Airline",
                     "perc_denied_boarding":"Percent Denied Boarding"
                 },
                title="Denied Boarding by Airline Carrier", color_discrete_sequence=px.colors.qualitative.Dark24)
fig.show()

In [256]:
denied_boarding_reshape = pd.melt(denied_boarding.rename(columns = {
    'num_passengers_denied_boarding_involuntarily_1':'Involuntary - Received Compensation', 
    'num_passengers_denied_boarding_involuntarily_2':'Involuntary - No Compensation', 
    'num_passengers_voluntarily_gave_up_seat':'Voluntary - Received Compensation'
    }), 
                                  id_vars = ['carrier', 'year', 'quarter'], 
                                  value_vars=['Involuntary - Received Compensation', 
                                              'Involuntary - No Compensation',
                                              'Voluntary - Received Compensation'
                                              ])


carrier_input = "Frontier Airlines"

fig = px.pie(denied_boarding_reshape[denied_boarding_reshape['carrier'] == carrier_input], 
             values='value', 
             names='variable', 
             labels = {
                 'value':'Total Passengers', 
                 'variable':'Denial Type'
             }, 
             title = "Denied Boarding by Type for " + carrier_input,
             color_discrete_sequence=px.colors.qualitative.Dark24)

# # plotly
fig = go.Figure()

# set up ONE trace
fig.add_trace(go.Scatter(x=df.index,
                         y=df[df.columns[0]],
                         visible=True)
             )

updatemenu = []
buttons = []

# button with one option for each dataframe
for col in df.columns:
    buttons.append(dict(method='restyle',
                        label=col,
                        visible=True,
                        args=[{'y':[df[col]],
                               'x':[df.index],
                               'type':'scatter'}, [0]],
                        )
                  )

# some adjustments to the updatemenus
updatemenu = []
your_menu = dict()
updatemenu.append(your_menu)

updatemenu[0]['buttons'] = buttons
updatemenu[0]['direction'] = 'down'
updatemenu[0]['showactive'] = True

# add dropdown menus to the figure
fig.update_layout(showlegend=False, updatemenus=updatemenu)
fig.show()
Share
Improve this answer
Follow



In [276]:

import numpy as np
import plotly.express as px

# generate data matching definition in question
data = pd.DataFrame(
    {
        "Category": np.random.choice(["One", "Two", "Three"], 100),
        "Sign": np.random.choice(list("+-"), 100),
    }
)

# simplify - use value_counts() on whole dataframe
df_vc = data.value_counts()
# create a figure for each category
figs = {
    c: px.pie(df_vc.loc[c].reset_index(), values=0, names="Sign").update_traces(
        name=c, visible=False
    )
    for c in df_vc.index.get_level_values("Category").unique()
}

# integrate figures per category into one figure
defaultcat = df_vc.index.get_level_values("Category").unique()[0]
fig = figs[defaultcat].update_traces(visible=True)
for k in figs.keys():
    if k != defaultcat:
        fig.add_traces(figs[k].data)

# finally build dropdown menu
fig.update_layout(
    updatemenus=[
        {
            "buttons": [
                {
                    "label": k,
                    "method": "update",
                    # list comprehension for which traces are visible
                    "args": [{"visible": [kk == k for kk in figs.keys()]}],
                }
                for k in figs.keys()
            ]
        }
    ]
)


In [320]:
df = denied_boarding_reshape[['carrier', 'variable','value']].set_index(['carrier', 'variable'])['value']

# create a figure for each category
figs = {
    c: px.pie(df.loc[c].reset_index(), values="value", names="variable", 
              labels = {
                 'value':'Total Passengers', 
                 'variable':'Denial Type'
             }, 
        color_discrete_sequence=px.colors.qualitative.Dark24).update_traces(
        name=c, visible=False, 
    )
    for c in df.index.get_level_values("carrier").unique()
}

# integrate figures per category into one figure
defaultcat = df.index.get_level_values("carrier").unique()[0]
fig = figs[defaultcat].update_traces(visible=True)
for k in figs.keys():
    if k != defaultcat:
        fig.add_traces(figs[k].data)

# finally build dropdown menu
fig.update_layout(
    updatemenus=[
        {
            "buttons": [
                {
                    "label": k,
                    "method": "update",
                    # list comprehension for which traces are visible
                    "args": [{"visible": [kk == k for kk in figs.keys()]},
                             {"title":go.layout.xaxis.Title(
            text=f"Denied Boardings by Airline Carrier <br><sup>{k} </sup>"
            )}],
                }
                for k in figs.keys()
            ]
        }
    ]
)


In [200]:
fig = px.area(denied_boarding_overall.rename(columns = {'perc_denied_boarding_involuntary':"Involuntary", "perc_denied_boarding_voluntary":"Voluntary"}), 
              x="year", y=["Voluntary","Involuntary"], 
                labels={
                    "year": "Year",
                     "value":"Total Passengers Denied Boarding", 
                     "variable":"Denial Type"
                 },
                title="Percent of Passengers Denied Boarding from 1990 to 2021", color_discrete_sequence=px.colors.qualitative.Dark24)
fig.update_xaxes(tickangle = 45)
fig.show()

In [192]:
fig = px.bar(denied_boarding_overall.rename(columns = { "voluntary":"Voluntary", 'involuntary':"Involuntary"}), 
             x="year", y=[ 'Voluntary', "Involuntary"], 
                labels={
                    "year": "Year",
                     "value":"Total Passengers Denied Boarding", 
                     "variable":"Denial Type"
                 },
                title="Total Passengers Denied Boarding from 1990 to 2021", color_discrete_sequence=px.colors.qualitative.Dark24)
fig.update_xaxes(tickangle = 45)
fig.show()